# Image Enhancement analysis
This code applies different image enhancement techniques based on classification ('good', 'bad', 'medium') and displays the original and transformed images along with their PSNR values.

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def negative_transformation(image):
    return 255 - image

def intensity_transformation(image, factor):
    return np.clip(factor * image, 0, 255).astype(np.uint8)

def log_transformation(image):
    c = 255 / np.log(1 + np.max(image))
    log_transformed = c * (np.log(image + 1))
    return np.array(log_transformed, dtype=np.uint8)

def gamma_correction(image, gamma):
    return np.array(255 * (image / 255) ** gamma, dtype=np.uint8)

def histogram_equalization(image):
    return cv2.equalizeHist(image)

def otsu_binarization(image):
    _, binarized_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binarized_image

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return "Infinity"
    max_pixel = 255.0
    psnr_value = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr_value

def enhance_image(image_path, classification):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    save_dir = os.path.dirname(image_path)
    
    if classification == 'good':
        negative_image = negative_transformation(image)
        intensity_transformed_image = intensity_transformation(image, 1.5)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_negative.png'), negative_image)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_intensity.png'), intensity_transformed_image)
        return image, [negative_image, intensity_transformed_image]

    elif classification == 'bad':
        log_transformed_image = log_transformation(image)
        gamma_corrected_image = gamma_correction(image, 2.0)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_log.png'), log_transformed_image)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_gamma.png'), gamma_corrected_image)
        return image, [log_transformed_image, gamma_corrected_image]

    elif classification == 'medium':
        intensity_transformed_image = intensity_transformation(image, 1.2)
        hist_equalized_image = histogram_equalization(image)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_intensity.png'), intensity_transformed_image)
        cv2.imwrite(os.path.join(save_dir, f'{base_filename}_histogram.png'), hist_equalized_image)
        return image, [intensity_transformed_image, hist_equalized_image]
    
    else:
        raise ValueError("Invalid classification. Must be 'good', 'bad', or 'medium'.")

def show_images(original_image, transformed_images, titles, psnr_values):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, len(transformed_images) + 1, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    for i, (transformed_image, title, psnr_value) in enumerate(zip(transformed_images, titles, psnr_values), start=2):
        plt.subplot(1, len(transformed_images) + 1, i)
        plt.imshow(transformed_image, cmap='gray')
        plt.title(f'{title}\nPSNR: {psnr_value:.2f} dB')
        plt.axis('off')
    
    plt.show()

def enhancement(src, classi):
    # Example usage:
    image_path = src
    classification = classi
    
    original_image, transformed_images = enhance_image(image_path, classification)
    
    binarized_images = [otsu_binarization(img) for img in transformed_images]
    psnr_values = [calculate_psnr(original_image, binarized_image) for binarized_image in binarized_images]
    
    if classification == 'good':
        show_images(original_image, transformed_images, ['Negative Transformation', 'Intensity Transformation'], psnr_values)
    elif classification == 'bad':
        show_images(original_image, transformed_images, ['Log Transformation', 'Gamma Correction'], psnr_values)
    elif classification == 'medium':
        show_images(original_image, transformed_images, ['Intensity Transformation', 'Histogram Equalization'], psnr_values)

# Example usage
# enhancement("path_to_image.png", "good")
